In [ ]:
import json
import random
random.seed(27)
from functools import partial
from collections import defaultdict
from multiprocessing import Pool
from tqdm import tqdm

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

In [ ]:
import cv2
import albumentations as A

In [ ]:
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from transformers import BertTokenizer

# Data

## Load csv

In [ ]:
path = '/kaggle/input/shopee-product-matching/'

In [ ]:
train_csv = pd.read_csv(path+'train.csv')
train_csv.head()

In [ ]:
train_img_name = train_csv['image']
train_img_path = path + 'train_images/' + train_csv['image']
train_title = train_csv['title']
train_label = train_csv['label_group']
train_hash = train_csv['image_phash']

In [ ]:
text_token_dims = 300

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
train_title_token = np.array(tokenizer(train_title.tolist(),
                                       padding='max_length',
                                       truncation=True , 
                                       max_length=text_token_dims)['input_ids'],
                             dtype = np.uint32)

# Test set

In [ ]:
def test_set(imagePath, label_group, test_size = 0.1):
  n_test = int(len(imagePath) * test_size)

  unique_label, count_label = np.unique(label_group, return_counts=True)
  labelWith2Samples = unique_label[count_label==2]
  idx = np.arange(len(labelWith2Samples))

  test_idx = idx[:n_test//2]## floor divide by 2, since each label has two image

  test_filter = np.zeros((len(imagePath), ), dtype=bool)
    
  path_index = np.arange(len(imagePath))

  gallery = []

  probe = []
  
  for label in labelWith2Samples[test_idx]:
    labelFilter = label_group == label
    test_filter = test_filter | labelFilter

    this_label_images = path_index[labelFilter]

    gallery.append(this_label_images[0])
    probe.append(this_label_images[1])
  return test_filter, gallery, probe


test_filter, test_gallery, test_probe = test_set(train_img_path, train_label)

In [ ]:
Gallery_label = train_label[test_gallery]
Probe_label = train_label[test_probe]

In [ ]:
Gallery_idx_dataset = tf.data.Dataset.from_tensor_slices(test_gallery)
Probe_idx_dataset = tf.data.Dataset.from_tensor_slices(test_probe)

In [ ]:
len(train_img_path)

In [ ]:
image_size = 224

def fetch(idx, image_size = image_size):
    image = cv2.resize(cv2.imread(train_img_path.iloc[idx])[...,[2,1,0]], dsize = (image_size,image_size), interpolation = cv2.INTER_AREA)
    title = train_title_token[idx]
    return image, title

def GetData(idx, size):
    image, title = tf.numpy_function(func=fetch, inp=[idx], Tout=[tf.uint8, tf.uint32])
    return {"image": image, "title": title}

Gallery = Gallery_idx_dataset.map(partial(GetData, size=image_size), num_parallel_calls=tf.data.AUTOTUNE).batch(500).prefetch(tf.data.AUTOTUNE)
Probe = Probe_idx_dataset.map(partial(GetData, size=image_size), num_parallel_calls=tf.data.AUTOTUNE).batch(500).prefetch(tf.data.AUTOTUNE)

In [ ]:
test_batch_g = next(iter(Gallery))

In [ ]:
test_batch_p = next(iter(Probe))

In [ ]:
def show_batch():
    num = 5
    plt.figure(figsize=(20,8))
    for i in range(num):
        plt.subplot(2,num,i+1)
        plt.imshow(test_batch_g['image'][i])
        plt.title('Gallery: ')
        plt.subplot(2,num,i+num+1)
        plt.imshow(test_batch_p['image'][i])
        plt.title('Probe: ')
    plt.show()
show_batch()

# Non Deep Model

## phash

In [ ]:
Gallery_hash = np.array([*map(lambda x: list(x), train_hash.iloc[test_gallery].to_numpy())])
Probe_hash = np.array([*map(lambda x: list(x), train_hash.iloc[test_probe].to_numpy())])

# Deep Model

In [ ]:
image_size = 224

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TripletLossLayer(layers.Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        
        anchor = tf.math.l2_normalize(anchor, axis=1)
        positive = tf.math.l2_normalize(positive, axis=1)
        negative = tf.math.l2_normalize(negative, axis=1)

        p_dist = tf.math.reduce_sum(tf.math.square(anchor-positive), axis=-1)
        n_dist = tf.math.reduce_sum(tf.math.square(anchor-negative), axis=-1)
        return tf.math.reduce_sum(tf.math.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

In [ ]:
tokenizer.vocab_size

# Model 1: Image

In [ ]:
def plot_loss(loss, val_loss, t):
    plt.figure(figsize = (10,5), facecolor = 'white')
    plt.plot(np.arange(1,len(loss)+1), loss, label = 'Train loss' )
    plt.plot(np.arange(1,len(val_loss)+1), val_loss, label = 'validation loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(t+': Training Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
loss = [7.67,6.23,5.76,5.39,5.02,4.75,4.52,4.38,
        3.82,3.76,3.64,3.57,3.45,3.32,3.21,3.12,
        3.13,3.07,2.98,2.95,2.93,2.92,2.86,2.81]
val_loss = [6.90,6.17,4.85,4.98,4.09,4.76,4.83,4.01,
            4.16,3.99,3.85,3.22,3.91,3.62,3.39,3.25,
            3.77,3.50,3.27,3.50,3.67,3.46,3.13,3.45]
plot_loss(loss, val_loss, 'Model3 (Image model)')

In [ ]:
def model_builder_Image(resolution, text_token_dims = text_token_dims, vocab_size = tokenizer.vocab_size, embedding_size = 256):
    ##IMAGE
    BackBone_img = keras.applications.EfficientNetB0(
        include_top=False, weights='imagenet',
        pooling = 'avg',
        input_tensor=layers.Input((resolution,resolution,3), name = 'image')
    )
    net_image_flatten = layers.Flatten()(BackBone_img.layers[-1].output)

    output = layers.Dense(embedding_size)(net_image_flatten)

    BackBone = keras.Model(inputs=[BackBone_img.layers[0].output], outputs=output, name = 'Image_Model')

    img_input_anchor = layers.Input((resolution,resolution,3), name='Anchor_img')
    img_input_positive = layers.Input((resolution,resolution,3), name='Positive_img')
    img_input_negative = layers.Input((resolution,resolution,3), name='Negative_img')

    anchor_embedding= BackBone([img_input_anchor])
    positive_embedding = BackBone([img_input_positive])
    negative_embedding = BackBone([img_input_negative])

    margin = 1
    loss_layer = TripletLossLayer(alpha=margin, name='triplet_loss_layer')([anchor_embedding, positive_embedding, negative_embedding])
    Triplet_Net = keras.Model(inputs=[img_input_anchor, img_input_positive, img_input_negative], outputs=loss_layer)
    Triplet_Net.compile(optimizer=keras.optimizers.RMSprop())
    return BackBone, Triplet_Net

Embedding_Net_Image, Triplet_Net_Image = model_builder_Image(image_size)

In [ ]:
Triplet_Net_Image.load_weights('../input/shopee-420-a2/Image-24-2.81-3.45.hdf5')

In [ ]:
Gallery_embedding_matrix_Image = Embedding_Net_Image.predict(Gallery)
Probe_embedding_matrix_Image = Embedding_Net_Image.predict(Probe)

In [ ]:
tf.keras.backend.clear_session()

# Model 2: Text

In [ ]:
loss = [30.45,20.99,17.62,14.79,12.48,11.28,10.00,9.47,8.18,7.82,
        7.40,6.70,6.27,6.07,5.66,5.35,5.05,4.89,4.70,4.36,
        4.38,4.24,4.02,3.95,3.68,3.66,3.52,3.37,3.35,3.20,
        3.14,3.00,2.89,2.88,2.84,2.79,2.65,2.67,2.53,2.47,
        2.48,2.40,2.39,2.31,2.33,2.24,2.23,2.17,2.10,2.11]
val_loss = [19.74,15.46,12.88,11.92,9.21,8.80,8.25,6.29,6.72,6.65,
            6.13,6.08,5.36,5.42,5.32,4.94,5.24,4.93,4.72,4.29,
            3.97,3.86,4.14,3.89,3.77,3.57,3.82,3.10,3.24,3.37,
            3.18,3.04,2.85,2.95,2.70,2.62,2.40,2.52,2.26,2.40,
            2.30,2.60,2.38,2.61,2.19,2.04,2.02,2.28,2.09,1.95
           ]
plot_loss(loss, val_loss, 'Model2 (Text model)')

In [ ]:
def model_builder_Text(resolution, text_token_dims = text_token_dims, vocab_size = tokenizer.vocab_size, embedding_size = 256):
    ##TEXT
    input_text_embed_dim = 70 # Embedding size for each token
    num_heads = 15 # Number of attention heads
    out_dim = 100 # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(text_token_dims,), name = 'title')
    embedding_layer = TokenAndPositionEmbedding(text_token_dims, vocab_size, input_text_embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(input_text_embed_dim, num_heads, out_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)

    #Connect two network
    output = layers.Dense(embedding_size)(x)

    BackBone = keras.Model(inputs=[inputs], outputs=output, name = 'SimpleTransformer')

    title_input_anchor = layers.Input((text_token_dims,), name='Anchor_title')
    title_input_positive = layers.Input((text_token_dims,), name='Positive_title')
    title_input_negative = layers.Input((text_token_dims,), name='Negative_title')

    anchor_embedding= BackBone([title_input_anchor])
    positive_embedding = BackBone([title_input_positive])
    negative_embedding = BackBone([title_input_negative])

    margin = 1
    loss_layer = TripletLossLayer(alpha=margin, name='triplet_loss_layer')([anchor_embedding, positive_embedding, negative_embedding])
    Triplet_Net = keras.Model(inputs=[title_input_anchor, title_input_positive,  title_input_negative], outputs=loss_layer)
    Triplet_Net.compile(optimizer=keras.optimizers.RMSprop())
    return BackBone, Triplet_Net

Embedding_Net_Text, Triplet_Net_Text = model_builder_Text(image_size)

Triplet_Net_Text.load_weights('../input/shopee-420-a2/text-50-2.11-1.95.hdf5')

In [ ]:
Gallery_embedding_matrix_Text = Embedding_Net_Text.predict(Gallery)
Probe_embedding_matrix_Text = Embedding_Net_Text.predict(Probe)

In [ ]:
tf.keras.backend.clear_session()

# Model 3: Image + Text

In [ ]:
loss = [7.38, 5.92, 5.20, 4.58, 4.02, 3.62, 3.18, 3.04, 
        2.79, 2.77, 2.54, 2.42, 2.34, 2.28, 2.23, 2.14, 
       2.0, 2.0, 1.94, 1.87, 1.78, 1.72, 1.77, 1.68]
val_loss = [7.48, 4.92, 4.02, 3.72, 3.73, 2.93, 2.64, 3.10, 
            3.01, 2.51, 2.45, 2.56, 2.58, 2.01, 2.04, 2.19,
           2.09, 1.84, 2.01, 1.7, 1.51, 1.48, 1.91, 1.63]
plot_loss(loss, val_loss, 'Model3 (Image + Text model)')

In [ ]:
def model_builder_Full(resolution, text_token_dims = text_token_dims, vocab_size = tokenizer.vocab_size, embedding_size = 256):
    ##IMAGE
    BackBone_img = keras.applications.EfficientNetB0(
        include_top=False, weights='imagenet',
        pooling = 'avg',
        input_tensor=layers.Input((resolution,resolution,3), name = 'image')
    )
    net_image_flatten = layers.Flatten()(BackBone_img.layers[-1].output)

    ##TEXT
    input_text_embed_dim = 70 # Embedding size for each token
    num_heads = 15 # Number of attention heads
    out_dim = 100 # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(text_token_dims,), name = 'title')
    embedding_layer = TokenAndPositionEmbedding(text_token_dims, vocab_size, input_text_embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(input_text_embed_dim, num_heads, out_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)

    #Connect two network
    net_concate = layers.Concatenate()([net_image_flatten, x])

    output = layers.Dense(embedding_size)(net_concate)

    BackBone = keras.Model(inputs=[BackBone_img.layers[0].output, inputs], outputs=output, name = 'EfficientNetB0_SimpleTransformer')


    img_input_anchor = layers.Input((resolution,resolution,3), name='Anchor_img')
    img_input_positive = layers.Input((resolution,resolution,3), name='Positive_img')
    img_input_negative = layers.Input((resolution,resolution,3), name='Negative_img')

    title_input_anchor = layers.Input((text_token_dims,), name='Anchor_title')
    title_input_positive = layers.Input((text_token_dims,), name='Positive_title')
    title_input_negative = layers.Input((text_token_dims,), name='Negative_title')


    anchor_embedding= BackBone([img_input_anchor, title_input_anchor])
    positive_embedding = BackBone([img_input_positive, title_input_positive])
    negative_embedding = BackBone([img_input_negative, title_input_negative])

    margin = 1
    loss_layer = TripletLossLayer(alpha=margin, name='triplet_loss_layer')([anchor_embedding, positive_embedding, negative_embedding])
    Triplet_Net = keras.Model(inputs=[img_input_anchor, title_input_anchor, img_input_positive, title_input_positive, img_input_negative, title_input_negative], outputs=loss_layer)
    Triplet_Net.compile(optimizer=keras.optimizers.RMSprop())
    return BackBone, Triplet_Net

Embedding_Net_Full, Triplet_Net_Full = model_builder_Full(image_size)


Triplet_Net_Full.load_weights('../input/shopee-420-a2/FULL-24-1.68-1.63.hdf5')

In [ ]:
Gallery_embedding_matrix_Full = Embedding_Net_Full.predict(Gallery)
Probe_embedding_matrix_Full = Embedding_Net_Full.predict(Probe)

In [ ]:
tf.keras.backend.clear_session()

# Result analysis

## CMC

In [ ]:
def L2_v2m(v, m):
    return np.sqrt(np.sum(np.square(v - m),axis = 1))

def cosine_similarity_v2m(v, m):
    return np.sum(v*m, axis = 1)/(np.linalg.norm(v) * np.linalg.norm(m, axis = 1))


def hammingDist_v2m(s, sm):
    return np.sum(s != sm, axis = 1)

In [ ]:
def ranked_histogram(Gallery_embedding_matrix, Probe_embedding_matrix, dist_measure = L2_v2m):
    ranked_histogram_ = np.zeros(len(Probe_label))
    
    top5 = []

    resuld_dict = {'posting_id': [], 'matches': []}
    for i in tqdm(range(len(Gallery_label))):
        dist_matrix = dist_measure(Gallery_embedding_matrix[i], Probe_embedding_matrix)

        sorted_dist = np.argsort(dist_matrix)

        ranked = Probe_label.iloc[sorted_dist]
        
        
        find_result = ranked == Gallery_label.iloc[i]
        
        
        if (np.any(find_result.iloc[:5])):
            top5.append(dist_matrix[sorted_dist[find_result][0]])
            
        match = np.where(ranked == Gallery_label.iloc[i])[0][0]

        ranked_histogram_[match] += 1
    return ranked_histogram_, np.array(top5)

def cmc(ranked_histogram):
    cmc = np.zeros(len(Probe_label))
    for i in range(len(Probe_label)):
        cmc[i] = np.sum(ranked_histogram[:(i + 1)])
    print(cmc)
    return cmc

In [ ]:
ranked_histogram_phash, top5_phash = ranked_histogram(Gallery_hash, Probe_hash, dist_measure = hammingDist_v2m)
ranked_histogram_text, top5_text = ranked_histogram(Gallery_embedding_matrix_Text, Probe_embedding_matrix_Text)
ranked_histogram_image, top5_image = ranked_histogram(Gallery_embedding_matrix_Image, Probe_embedding_matrix_Image)
ranked_histogram_full, top5_full = ranked_histogram(Gallery_embedding_matrix_Full, Probe_embedding_matrix_Full)

In [ ]:
cmc_phash = cmc(ranked_histogram_phash)
cmc_text = cmc(ranked_histogram_text)
cmc_image = cmc(ranked_histogram_image)
cmc_full = cmc(ranked_histogram_full)

In [ ]:
fig = plt.figure(figsize=[20, 8])
ax = fig.add_subplot(1, 2, 1)
ax.plot(cmc_phash, label = 'phash')
ax.plot(cmc_image, label = 'Image')
ax.plot(cmc_text, label = 'Text')
ax.plot(cmc_full, label = 'Full')
ax.set_xlabel('Rank')
ax.set_ylabel('Count')
ax.set_title('CMC Curve')
ax.legend(loc = 'lower right')
ax = fig.add_subplot(1, 2, 2)
ax.plot(cmc_phash/len(Gallery_label), label = 'phash')
ax.plot(cmc_image/len(Gallery_label), label = 'Image')
ax.plot(cmc_text/len(Gallery_label), label = 'Text')
ax.plot(cmc_full/len(Gallery_label), label = 'Full')
ax.set_xlabel('Rank')
ax.set_ylabel('Count')
ax.set_title('CMC Curve scaled')
ax.legend(loc = 'lower right')

plt.show()

In [ ]:
def accuracy(cmc):
    print('Top 1 accuracy: {:1f}'.format((cmc/len(Gallery_label))[0]))
    print('Top 5 accuracy: {:1f}'.format((cmc/len(Gallery_label))[4]))
    
print('Phash: ')
accuracy(cmc_phash)
print('Image: ')
accuracy(cmc_image)
print('Text: ')
accuracy(cmc_text)
print('Full: ')
accuracy(cmc_full)

## possitive, negative distance

In [ ]:
def pos_neg_dist(Gallery_embedding_matrix, Probe_embedding_matrix, dist_measure = L2_v2m):
    ranked_histogram_ = np.zeros(len(Probe_label))
    positive_pair = []
    negative_pair = []
    
    

    for i in tqdm(range(len(Gallery_label))):
        dist_matrix = dist_measure(Gallery_embedding_matrix[i], Probe_embedding_matrix)

        positive_pair.append(dist_matrix[i])
        
        
        mask = np.ones(dist_matrix.shape, dtype=bool)
        mask[i] = False
        
        negative_pair += dist_matrix[mask].tolist()
        
    return positive_pair, negative_pair



In [ ]:
positive_pair, negative_pair = pos_neg_dist(Gallery_hash, Probe_hash, dist_measure = hammingDist_v2m)
sns.histplot(positive_pair, kde=True, stat = 'density', color = 'red')
sns.histplot(negative_pair, kde=True, stat = 'density', color = 'blue')

In [ ]:
positive_pair, negative_pair = pos_neg_dist(Gallery_embedding_matrix_Text, Probe_embedding_matrix_Text)
sns.histplot(positive_pair, kde=True, stat = 'density', color = 'red')
sns.histplot(negative_pair, kde=True, stat = 'density', color = 'blue')

In [ ]:
positive_pair, negative_pair = pos_neg_dist(Gallery_embedding_matrix_Image, Probe_embedding_matrix_Image)
sns.histplot(positive_pair, kde=True, stat = 'density', color = 'red')
sns.histplot(negative_pair, kde=True, stat = 'density', color = 'blue')

In [ ]:
positive_pair, negative_pair = pos_neg_dist(Gallery_embedding_matrix_Full, Probe_embedding_matrix_Full)
sns.histplot(positive_pair, kde=True, stat = 'density', color = 'red')
sns.histplot(negative_pair, kde=True, stat = 'density', color = 'blue')

## Top5 distance

In [ ]:
sns.displot(top5_phash, kde=True)

In [ ]:
sns.displot(top5_text, kde=True)

In [ ]:
sns.displot(top5_image, kde=True)

In [ ]:
sns.displot(top5_full, kde=True)

## TSNE

In [ ]:
# tsne_embeddings = TSNE(random_state=4).fit_transform(np.concatenate((Gallery_embedding_matrix_Full, Probe_embedding_matrix_Full), axis = 0))
# fig = plt.figure(figsize=[12, 12])
# ax = fig.add_subplot(1, 1, 1)
# ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c = np.concatenate((Gallery_label, Probe_label), axis = 0));
# ax.set_title('Full model')

In [ ]:
tsne_embeddings = TSNE(random_state=4).fit_transform(np.concatenate((Gallery_embedding_matrix_Text, Probe_embedding_matrix_Text), axis = 0))
fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(1, 1, 1)
fig = plt.figure(figsize=[12, 12])
ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c = np.concatenate((Gallery_label, Probe_label), axis = 0));
ax.set_title('Text model')

In [ ]:
tsne_embeddings = TSNE(random_state=4).fit_transform(np.concatenate((Gallery_embedding_matrix_Image, Probe_embedding_matrix_Image), axis = 0))
fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(1, 1, 1)
ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c = np.concatenate((Gallery_label, Probe_label), axis = 0));
ax.set_title('Image model')

In [ ]:
tsne_embeddings = TSNE(random_state=4).fit_transform(np.concatenate((Gallery_embedding_matrix_Full, Probe_embedding_matrix_Full), axis = 0))
fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(1, 1, 1)
ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c = np.concatenate((Gallery_label, Probe_label), axis = 0));
ax.set_title('Full model')

## Error analysis
if there is image that closer than similar image, plot those image. if more than 5, plot first 5 only

In [ ]:
def ranked_histogram(idx, Gallery_embedding_matrix, Probe_embedding_matrix, dist_measure = L2_v2m):
    target_idx = idx
    
    plt.figure(figsize=(35,5), facecolor = 'white')
    plt.subplot(1,7,1)
    plt.imshow(cv2.imread(train_img_path.iloc[test_gallery[idx]])[...,[2,1,0]])
    plt.title('ancher')
    
    distance = L2_v2m(Gallery_embedding_matrix[idx], Probe_embedding_matrix)
    
    sorted_dist = np.argsort(distance)
    
    ranked = Probe_label.iloc[sorted_dist]
    
    match = np.where(ranked == Gallery_label.iloc[idx])[0][0]
    
    plt.subplot(1,7,2)
    plt.imshow(cv2.imread(train_img_path.iloc[test_probe[idx]])[...,[2,1,0]])
    plt.title('excepted: {:2f}, found at {}'.format(distance[idx], match))
    
    if match > 5:
        match = 5
    
    plot_idx = 3
    for i in range(match):
        plt.subplot(1,7,plot_idx)
        plt.imshow(cv2.imread(train_img_path.iloc[test_probe[sorted_dist[i]]])[...,[2,1,0]])
        plt.title('unmatch: {:2f}'.format(distance[sorted_dist[i]]))
        plot_idx += 1
    plt.plot()

    

In [ ]:
for i in range(5):
    ranked_histogram(i, Gallery_embedding_matrix_Full, Probe_embedding_matrix_Full)